In [3]:
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath("..")  # Adjust if necessary

# Add to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

In [4]:
import sys
print(sys.executable)

/Users/cesarlindberg/anaconda3/envs/cardiac_pinn/bin/python


In [5]:
%load_ext autoreload

In [6]:
%autoreload 2

In [7]:
# ---------------------------------------------
# 0) Imports & Setup
# ---------------------------------------------
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import numpy as np

from src.training.pinnex_data_loader import get_dataloaders
from src.networks.pinnex_models import PINNWithECG
from src.networks.pinn_wrapper import CachedPINNWrapper
from src.training.trainer_pinnex import train_pinnex_minibatch_with_ecg
from src.utils.evaluation import evaluate_pinnex, compute_metrics


In [19]:
# ---------------------------------------------
# 1) Load and Inspect Data
# ---------------------------------------------

n_epochs = 10
learning_rate = 1e-3
weight_decay = 1e-7
physics_weight = 0
batch_size = 100
batches_per_epoch = 1
phys_batch_size = 2048
val_ratio = 0.15
val_every_n_epochs = 1

In [15]:
# Paths for spatiotemporal and ECG data
spatiotemp_path = "../data/synthetic/parsed/activation_heart_123fib.parquet"
ecg_path = "../data/synthetic/parsed/ecg_heart_123fib.parquet"


train_loader, val_loader = get_dataloaders(
    spatiotemp_path=spatiotemp_path,
    ecg_path=ecg_path,
    batch_size=batch_size,
    val_ratio=val_ratio,
    seed=41,
    group_by_delay_variant=True,
    device="cpu"
)

Splitting by base_sim_id (delay variant grouping active, using seed 41).
Total unique base_sim_ids: 1605
Number of base_sim_ids for validation: 240
Validation base_sim_ids: [1275, 3122, 3307, 3471, 3656, 1387, 3628, 3589, 3501, 2056, 3042, 2152, 2333, 1305, 3757, 3702, 3562, 3638, 2112, 2372, 3418, 1062, 1029, 3530, 3616, 3576, 3124, 1070, 3389, 3217, 1313, 1399, 2096, 3099, 1016, 2110, 3113, 3594, 2111, 2072, 1082, 1243, 3679, 1036, 3597, 3773, 2241, 3568, 3143, 1318, 2291, 2377, 3467, 3382, 3249, 2084, 3344, 2182, 3516, 1126, 3427, 2192, 3136, 3555, 1250, 1391, 1237, 2, 2030, 3480, 3227, 3432, 3318, 1106, 2346, 3563, 3436, 3173, 2296, 3512, 3689, 2183, 2059, 2209, 1356, 3091, 2118, 1225, 2344, 3272, 1028, 3699, 3192, 3132, 2237, 3410, 3654, 3089, 3170, 3041, 3722, 3315, 1229, 3345, 1280, 1133, 2128, 1370, 3473, 1165, 2073, 2369, 1219, 3466, 3715, 2136, 3479, 1101, 1375, 2113, 2232, 1161, 3513, 3564, 1079, 3711, 1241, 2225, 3670, 3152, 2394, 1096, 3015, 3578, 2023, 2278, 3788, 1152, 2

In [16]:
params = {"v_max": 810, "t_scale": 215, "l_scale": 50000}

In [25]:


base_model = PINNWithECG(
    params,  # your existing params dictionary
    pde_latent_dim=64,
    ecg_latent_dim=64,  # Can be 64 if you want it smaller than pde_latent_dim for gated
    pde_hidden_architecture=[64, 128, 128], # Input 51 -> 128 -> 256 -> 256 (then final map to pde_latent_dim)
    ecg_hidden_architecture=[256, 128], # FC layers after CNN, input ~3328 (from 256 channels * 13 seq_len) -> 512 -> 256 (then final map to ecg_latent_dim)
    decoder_hidden_architecture=[128, 64, 32], # Input pde_latent_dim (128) -> 256 -> 128 -> 64 (then final map to 2 outputs)
    desired_channels_per_block=[16,16,32,32],
    n_leads=12,
    seq_len=201,
    fusion_mode="mul", # CHANGED based on your text's findings
    n_blocks=4  # Number of CNN blocks in ECG_Encoder
)


base_model = PINNWithECG(
    params,
    pde_latent_dim=15,      # adjust as needed
    ecg_latent_dim=15,
    pde_hidden_architecture=[15, 15, 15, 15, 15, 15, 15],  # Example hidden layer sizes for PDE encoder
    ecg_hidden_architecture=[128, 64],  # Example hidden layer sizes for ECG encoder (FC part)
    decoder_hidden_architecture=[32, 32],  # Example hidden layer sizes for decoder
    desired_channels_per_block=[16,16,32,32],
    n_leads=12,
    seq_len=201,
    fusion_mode="mul",
    n_blocks=4
)

#base_model.load_state_dict(torch.load("models/heart3fibTEST.pth"))
#checkpoint = torch.load("models/heart_123fib_phys.pth", map_location=torch.device('cuda'))

if False: # Your existing conditional block
    component_names = ["pde_encoder", "ecg_encoder", "decoder_body", "decoder_out"]

    for name in component_names:
        prefix = f"{name}."
        weights = {k.replace(prefix, ""): v for k, v in checkpoint.items() if k.startswith(prefix)}
        
        if weights:
            try:
                getattr(base_model, name).load_state_dict(weights)
                print(f"Successfully loaded weights into base_model.{name}")
            except RuntimeError as e:
                print(f"Error loading weights for base_model.{name}: {e}. Not loaded.")
        else:
            print(f"No weights found for '{name}' in checkpoint.")

    # --- Load Gate weights (if applicable) ---
    if base_model.fusion_mode == "gated":
        if hasattr(base_model, 'gate'):
            gate_weights = {k.replace("gate.", ""): v for k, v in checkpoint.items() if k.startswith("gate.")}
            if gate_weights:
                try:
                    base_model.gate.load_state_dict(gate_weights)
                    print("Successfully loaded weights into base_model.gate")
                except RuntimeError as e:
                    print(f"Error loading weights for base_model.gate: {e}. Not loaded.")
            else:
                print("No weights found for 'gate' in checkpoint.")
        else:
            print("'gate' layer expected but not found in base_model. Skipping gate weight loading.")


model = CachedPINNWrapper(base_model).to("cpu")


In [27]:

# ---------------------------------------------
# 5) Train the PINNEX
# ---------------------------------------------
train_loss, val_loss, data_loss, pde_loss = train_pinnex_minibatch_with_ecg(
    wrapper=model,
    train_loader=train_loader,
    val_loader=val_loader,
    params=params,
    n_epochs=n_epochs-5,
    lr=learning_rate,
    weight_decay=1e-8,
    physics_weight=0,
    normal_enforcement_weight=0, # New parameter
    cv_reg_weight=0,
    batches_per_epoch=batches_per_epoch,
    phys_batch_size=1024,
    val_every_n_epochs=100,
    device='cpu',  # or 'cuda' if available
    collocation_points_filepath="fine_heart_coll_pts_normals.txt"
)


KeyboardInterrupt: 

In [1]:
epochs = range(1, len(train_loss) + 1)
val_epochs = np.arange(val_every_n_epochs, len(train_loss) + 1, val_every_n_epochs)
plt.figure(figsize=(10, 4))
    
# Plot Total Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Train Loss')
plt.scatter(val_epochs, val_loss, color='red', label='Val Loss', zorder=3)
plt.yscale("log")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss (log scale)")
plt.title("Total Loss vs. Epoch")
# Plot Data vs PDE Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, data_loss, label='Data Loss')
plt.plot(epochs, pde_loss, label='PDE Residual Loss')
plt.yscale("log")
plt.xlabel("Epoch")
plt.ylabel("Loss (log scale)")  
plt.legend()
plt.title("Data vs PDE Loss")

plt.tight_layout()
plt.show()


NameError: name 'train_loss' is not defined

In [ ]:
# Run Evaluation
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Evaluate model on validation set
true_T, pred_T, pred_c = evaluate_pinnex(model, val_loader, device)

# Compute metrics for activation time T
re_T, mse_T, cc_T = compute_metrics(true_T, pred_T)

# Print results
print("------ Evaluation Results ------")
print(f"Relative Error (RE) for T: {re_T:.4f}")
print(f"Mean Squared Error (MSE) for T: {mse_T:.4f}")
print(f"Correlation Coefficient (CC) for T: {cc_T:.4f}")


In [ ]:
# Save the trained model
torch.save(model.state_dict(), "models/eik_testing.pth")
# TO LOAD
#model.load_state_dict(torch.load("pinn_mod1.pth"))
#model.eval()

In [722]:
from src.utils.igb_utils import write_pinnex_predictions_streamed

write_pinnex_predictions_streamed(base_model, 
                                  block_pts_file="../data/synthetic/raw/heart_1200.pts", 
                                  ecg_parquet_file="../data/synthetic/parsed/ecg_case0004.parquet",
                                  sim_id=1000,
                                  filename='case0004_predby_123fib_phys.dat', 
                                  CV=False
                                 )
                          
#igbhead -x11767 -y1 -z1 -t5001 -dfloat --create -f vm_phys.igb predictions_PINN.txt to convert to igb

Starting PINN-ECG prediction process...
Reading block.pts file: ../data/synthetic/raw/heart_1200.pts...
Loaded 207079 spatial points.
Loading ECG data for sim_id 1000 from ../data/synthetic/parsed/ecg_case0004.parquet...
Loaded ECG data with shape torch.Size([12, 200]).
Streaming spatial data and running inference...
Prediction process completed. Results written to case0004_predby_123fib_phys.dat
Creating IGB file with command: igbhead -x207079 -y1 -z1 -t1 -dfloat -slittle -X1.3856e-05 -Y1 -Z1 -T100 -I1.91988e-10 -J1 -K1 -L1 -S1 -1"um" -2"um" -3"um" -4"ms" --create -fcase0004_predby_123fib_phys.igb case0004_predby_123fib_phys.dat
Successfully created IGB file: case0004_predby_123fib_phys.igb
Temporary prediction file case0004_predby_123fib_phys.dat removed.
Process complete.


Adjusting dim_x to make dimensions consistent
Adjusting dim_y to make dimensions consistent
Adjusting dim_t to make dimensions consistent


In [855]:
evaluation_results = {}

In [921]:
import os
import torch

from notebooks.evaluation.evaluation_pipeline import evaluate_fibrosis_detection

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pinn_model_for_evaluation = base_model # <<--- REPLACE THIS WITH YOUR LOADED MODEL INSTANCE

# 2. Define paths (use absolute paths for robustness)
notebook_base_dir = os.getcwd() # Assumes notebook is in 'notebooks/'

healthy_parquet_sim_id = 16 # Integer or string, e.g., from your example 1000, 10, etc.

# Paths relative to the NOTEBOOK'S location
folder = "heart_1fib"
ecg_file_rel = "evaluation/" + folder + "/ecg_all.parquet" # ADJUST
pts_file_rel = "evaluation/" + folder + "/heart.pts" # ADJUST - ensure this is the correct .pts for your sims
delay_file_rel = "evaluation/" + folder + "/delays.txt"
sim_raw_dir_rel = "evaluation/" + folder # ADJUST

ecg_parquet_abs = os.path.abspath(os.path.join(notebook_base_dir, ecg_file_rel))
block_pts_abs = os.path.abspath(os.path.join(notebook_base_dir, pts_file_rel))
delay_file_abs = os.path.abspath(os.path.join(notebook_base_dir, delay_file_rel))
sim_raw_base_abs = os.path.abspath(os.path.join(notebook_base_dir, sim_raw_dir_rel))

model_tag_name = "heart_1fib_phys" # For naming output files

# 3. Run the evaluation
# Ensure 'pinn_model_for_evaluation' is your loaded and ready-to-use model object
if pinn_model_for_evaluation is not None: # Check if model loaded
    evaluation_results[model_tag_name] = evaluate_fibrosis_detection(
        model=pinn_model_for_evaluation,
        healthy_parquet_sim_id=healthy_parquet_sim_id,
        ecg_parquet_file_abs_path=ecg_parquet_abs,
        block_pts_file_abs_path=block_pts_abs,
        delay_file_abs_path=delay_file_abs,
        sim_raw_base_abs_path=sim_raw_base_abs,
        model_name_tag=model_tag_name,
        CV_predictions=False,  # Set to True if your model output for 'T_pred' should come from 'c_pred'
        prediction_batch_size=1024,
        force_regenerate_predictions=False, # Set to True if you modified the model or want fresh predictions
        force_regenerate_diffs=False      # Set to True if you want to re-calculate IGB differences
    )
else:
    print("Model not loaded. Please load your model before running evaluation.")



--- Evaluation: heart_1fib_phys | Healthy Parquet ID: 16 ---
Loaded delay file with 44 lines.

Processing FIBROTIC: Parquet ID 180
  Diff Metrics - PCC: 0.9885, CosineSim(Diff): 0.9853
  Direct Fibrotic PCC: 0.9998
  Direct Fibrotic CosineSim: 0.9981
progress: 1/40

Processing FIBROTIC: Parquet ID 181
  Diff Metrics - PCC: 0.9893, CosineSim(Diff): 0.9837
  Direct Fibrotic PCC: 0.9998
  Direct Fibrotic CosineSim: 0.9967
progress: 2/40

Processing FIBROTIC: Parquet ID 182
  Diff Metrics - PCC: 0.9824, CosineSim(Diff): 0.9724
  Direct Fibrotic PCC: 0.9997
  Direct Fibrotic CosineSim: 0.9998
progress: 3/40

Processing FIBROTIC: Parquet ID 183
  Diff Metrics - PCC: 0.9824, CosineSim(Diff): 0.9685
  Direct Fibrotic PCC: 0.9997
  Direct Fibrotic CosineSim: 0.9998
progress: 4/40

Processing FIBROTIC: Parquet ID 114
  Diff Metrics - PCC: 0.9915, CosineSim(Diff): 0.9924
  Direct Fibrotic PCC: 0.9998
  Direct Fibrotic CosineSim: 0.9990
progress: 5/40

Processing FIBROTIC: Parquet ID 115
  Diff Me

In [945]:
results = evaluation_results["heart_123fib_phys"]
sim_ids = results["fibrotic_parquet_sim_ids"]
individual_diff_pcc = results["individual_diff_pcc"]
individual_diff_cosine = results["individual_diff_cosine"]
individual_direct_fib_pcc = results["individual_direct_fib_pcc"]
individual_direct_fib_cosine = results["individual_direct_fib_cosine"]


# Initialize accumulators and counter
total_diff_pcc = 0
total_diff_cosine = 0
total_direct_fib_pcc = 0
total_direct_fib_cosine = 0
count = 0

# Loop through sim_ids and accumulate metrics for sim_id < 199
for i in range(len(sim_ids)):
    sim_id = int(sim_ids[i])
    
    if int(str(sim_id)[0]) != 3:
        continue

        
    total_diff_pcc += individual_diff_pcc[i]
    total_diff_cosine += individual_diff_cosine[i]
    total_direct_fib_pcc += individual_direct_fib_pcc[i]
    total_direct_fib_cosine += individual_direct_fib_cosine[i]
    count += 1
print(count)
# Calculate averages
avg_diff_pcc = total_diff_pcc / count
avg_diff_cosine = total_diff_cosine / count
avg_direct_fib_pcc = total_direct_fib_pcc / count
avg_direct_fib_cosine = total_direct_fib_cosine / count

# Print results
print("--- Averaged Results ---")
print(f"avg_diff_pcc: {avg_diff_pcc:.4f}")
print(f"avg_diff_cosine: {avg_diff_cosine:.4f}")
print(f"avg_direct_fib_pcc: {avg_direct_fib_pcc:.4f}")
print(f"avg_direct_fib_cosine: {avg_direct_fib_cosine:.4f}")
print("--- Evaluation Complete ---")


40
--- Averaged Results ---
avg_diff_pcc: 0.5678
avg_diff_cosine: 0.7547
avg_direct_fib_pcc: 0.9888
avg_direct_fib_cosine: 0.9968
--- Evaluation Complete ---


In [922]:
import numpy as np
from scipy import stats

def fisher_z(r):
    """Clip to open interval (-1,1) to avoid inf, then Fisher transform."""
    return np.arctanh(np.clip(r, -0.999999, 0.999999))

def paired_ci_and_test(modelA_vals, modelB_vals, alpha=0.05):
    """
    Paired comparison of two models tested on the *same* samples.

    Returns:
        mean_diff_r : mean(B − A) on the original r scale
        ci_r        : tuple (low, high) CI on r scale
        p_t         : p-value from paired t-test
        p_wilcoxon  : p-value from Wilcoxon signed-rank (non-parametric)
    """
    a = np.asarray(modelA_vals, dtype=float)
    b = np.asarray(modelB_vals, dtype=float)

    # Drop pairs where either value is nan
    mask = ~np.isnan(a) & ~np.isnan(b)
    a, b = a[mask], b[mask]
    if len(a) < 2:
        return np.nan, (np.nan, np.nan), np.nan, np.nan

    # Fisher transform
    z_a, z_b = fisher_z(a), fisher_z(b)
    diff_z   = z_b - z_a

    # Mean & CI on z
    mean_z   = diff_z.mean()
    sem_z    = stats.sem(diff_z)
    df       = len(diff_z) - 1
    t_crit   = stats.t.ppf(1 - alpha/2, df)
    ci_z = (mean_z - t_crit * sem_z, mean_z + t_crit * sem_z)

    # Back-transform CI and mean to r scale (addend interpretation)
    mean_r  = np.tanh(mean_z)
    ci_r    = tuple(np.tanh(ci_z))

    # Significance tests
    t_stat, p_t        = stats.ttest_rel(z_b, z_a)   # works in z too
    w_stat, p_wilcoxon = stats.wilcoxon(diff_z)      # signed-rank

    return mean_r, ci_r, p_t, p_wilcoxon
    
name = "heart_1fib"
res_A = evaluation_results[name]
res_B = evaluation_results[name + "_phys"]
metrics = ["individual_diff_pcc",
           "individual_diff_cosine",
           "individual_direct_fib_pcc",
           "individual_direct_fib_cosine"]

for m in metrics:
    mean_gain, ci_gain, p_t, p_w = paired_ci_and_test(res_A[m], res_B[m])
    print(f"{m}:  Δ̄ = {mean_gain:+.3f}  "
          f"95 % CI = ({ci_gain[0]:+.3f}, {ci_gain[1]:+.3f})  "
          f"p_t = {p_t:.4g}  p_w = {p_w:.4g}")


individual_diff_pcc:  Δ̄ = +0.097  95 % CI = (+0.035, +0.159)  p_t = 0.0031  p_w = 0.00369
individual_diff_cosine:  Δ̄ = +0.177  95 % CI = (+0.121, +0.233)  p_t = 2.043e-07  p_w = 1.162e-06
individual_direct_fib_pcc:  Δ̄ = +0.126  95 % CI = (+0.075, +0.176)  p_t = 1.246e-05  p_w = 5.846e-05
individual_direct_fib_cosine:  Δ̄ = +0.025  95 % CI = (+0.010, +0.039)  p_t = 0.001302  p_w = 0.0002806


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters for each component
pde_encoder_params = count_parameters(base_model.pde_encoder)
ecg_encoder_params = count_parameters(base_model.ecg_encoder)

print(f"PDE Encoder Parameters: {pde_encoder_params}")
print(f"ECG Encoder Parameters: {ecg_encoder_params}")
print(f"Total Model Parameters: {pde_encoder_params + ecg_encoder_params}")

# Print the model's weights
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Size: {param.size()}) # | Values: \n{param.data}\n")
